In [ ]:
import os
os.chdir("/Users/mariusmahiout/Documents/repos/ising_core/python")

from src.sampler_sens import run_sensitivity_analysis

os.chdir("..")

In [ ]:
run_sensitivity_analysis(
    path="./analyses/sampler_sensitivity/sampler_sensitivity.pdf"
    num_runs = 30,
    min_units = 3,
    max_units = 300,
    beta = 1.3,
    num_sims = 15_000,
    num_burn = 1000,
    beta_min = 0.1,
    beta_max = 10,
    num_units = 50,
    sims_min = 1000,
    sims_max = 500_000,
)